In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import HTMLParser as htm
import string
import re

# SK-learn library for splitting data
from sklearn.model_selection import train_test_split

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

/Library/Python/2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Python/2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Read in data

In [39]:
data = pd.read_csv("tweet_data_1.csv",sep='\t',quoting=3)
data["escape"] = data.apply(lambda row: htm.HTMLParser().unescape(row[1].decode("utf-8")),axis=1)

data.head()

,Id,Tweet,Emotion,Positive,escape
0,138881940341260288:,I got a surprise for all you bitches...pull th...,:: surprise,0,I got a surprise for all you bitches...pull th...
1,144479819843911683:,If I was a thief.. The first thing I would ste...,:: joy,1,If I was a thief.. The first thing I would ste...
2,139110849120972800:,"""&quot;@RevRunWisdom: not afraid of tomorrow, ...",:: fear,0,"""""@RevRunWisdom: not afraid of tomorrow, for I..."
3,141532076791971840:,"""Extreme can neither fight nor fly.&#xA;-- Wil...",:: fear,0,"""Extreme can neither fight nor fly.\n-- Willia..."
4,145353048817012736:,Thinks that @melbahughes had a great 50th birt...,:: surprise,0,Thinks that @melbahughes had a great 50th birt...


[':: joy' ':: joy' ':: fear' ..., ':: joy' ':: surprise' ':: sadness']


## Split data into train & test sets

In [70]:
# Train and test data frames
train, test = train_test_split(data, test_size = 0.2)

# Train and test target labels
train_pol_y = train.ix[:,3].tolist()
test_pol_y = test.ix[:,3].tolist()

#binarize emo labels
from sklearn import preprocessing
train_emo = train.ix[:,2].tolist()
test_emo = test.ix[:,2].tolist()

emo_bin = preprocessing.LabelBinarizer()

train_emo_y = emo_bin.fit_transform(train_emo)
tests_emo_y = emo_bin.transform(test_emo)
# To get emotionas use emo_bin.inverse_transform(tests_emo_y)


# MC NEW CODE
# Convert to arrays
# train_pol_y = np.asarray(train_pol_y)
# test_pol_y = np.asarray(test_pol_y)

# MC NEW CODE
# Train and test x
train_pol_x = train.ix[:, 4].tolist()
test_pol_x = test.ix[:, 4].tolist()
# train_pol_x
# MC NEW CODE
# Convert to arrays
# train_pol_x = np.asarray(train_pol_x)
# test_pol_x = np.asarray(test_pol_x)

## Manually preprocess data

In [6]:
def process_data(data):
    """Converts to lowercase, strips out punctuation,
    removes excess whitespace within a string & leading & trailing whitespace"""
    new_list = []
    table = string.maketrans("","")
    for elem in data:
        elem = "".join(i for i in elem if ord(i)<128)
        elem = str(elem)        
        elem = elem.lower()
        elem = elem.translate(table, string.punctuation)
        
        # Comment these 2 lines out to improve positive
        elem = re.sub(' +',' ', elem)
        elem = elem.strip()
        
        new_list.append(elem)
    return new_list

train_pol_x = process_data(train_pol_x)
test_pol_x = process_data(test_pol_x)
# train_pol_x

## Exploratory on entire dataset

In [23]:
print "Dataset size: %s \n" % len(data)

print "Polarity counts"
print data.Positive.value_counts()
print "\n"
#avg number of words

print "Avg Words %s \n" % np.mean([len(s.split(" ")) for s in data.escape])
print "Emotion Counts"
print data.Emotion.value_counts()

Dataset size: 21051 

Polarity counts
0    12811
1     8240
Name: Positive, dtype: int64


Avg Words 16.317277089 

Emotion Counts
:: joy         8240
:: surprise    3849
:: sadness     3830
:: fear        2816
:: anger       1555
:: disgust      761
Name: Emotion, dtype: int64


In [28]:
#top words
from collections import Counter
holder = Counter()

#process entire dataset
data_ = data.ix[:, 4].tolist()
process_data(data_)

for i in data_:
    for word in i.split(" "):
        holder[word] += 1

In [34]:
from nltk.corpus import stopwords
for i in holder.most_common(10):
    #if i[0].lower() not in stopwords.words("english"):
    print "%s \t %s" % (i[0],i[1])

 	 17280
the 	 8174
to 	 7879
I 	 7157
a 	 5831
and 	 5002
my 	 4751
of 	 4661
in 	 3882
is 	 3508


## Model with preprocessed data

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

#min_df_options = [1, 2, 5, 10, 25]

#for df in min_df_options:
#print "Minimum DF", df
vectorizer = TfidfVectorizer(min_df=2,
                             use_idf=True,
                            stop_words='english',
                            )
train_vectors = vectorizer.fit_transform(train_pol_x)
test_vectors = vectorizer.transform(test_pol_x)

base1 = svm.SVC(kernel='linear')
base1.fit(train_vectors, train_pol_y)
predict_base1 = base1.predict(test_vectors)

target_names = ["Negative","Positive"]
print classification_report(test_pol_y,predict_base1, target_names = target_names)

             precision    recall  f1-score   support

   Negative       0.78      0.84      0.81      2565
   Positive       0.72      0.62      0.67      1646

avg / total       0.75      0.76      0.75      4211



[0 0 1 ..., 1 0 1]


## Model without preprocessed data

### Try different hyperparams

In [21]:
#https://marcobonzanini.com/2015/01/19/sentiment-analysis-with-python-and-scikit-learn/
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

min_df_options = [1, 2, 5]
strip_accents_options = ['ascii', 'unicode']
lowercase_options = [True, False]

for elem in strip_accents_options:
    print "ACCENT:", elem
    for op in lowercase_options:
        print "LOWERCASE OPTION:", str(op)
        for df in min_df_options:
            print "Minimum DF:", df
            vectorizer = TfidfVectorizer(min_df=df,
                                         use_idf=True,
                                        stop_words='english',
                                         strip_accents=elem,
                                         lowercase=op
                                        )
            train_vectors = vectorizer.fit_transform(train.ix[:,4].tolist())
            test_vectors = vectorizer.transform(test.ix[:,4].tolist())

            base1 = svm.SVC(kernel='linear')
            base1.fit(train_vectors, train_pol_y)
            predict_base1 = base1.predict(test_vectors)

            target_names = ["Negative","Positive"]
            print classification_report(test_pol_y,predict_base1, target_names = target_names)


ACCENT: ascii
LOWERCASE OPTION: True
Minimum DF: 1
             precision    recall  f1-score   support

   Negative       0.80      0.85      0.82      2598
   Positive       0.73      0.65      0.69      1613

avg / total       0.77      0.77      0.77      4211

LOWERCASE OPTION: False
Minimum DF: 1
             precision    recall  f1-score   support

   Negative       0.79      0.85      0.82      2598
   Positive       0.72      0.65      0.68      1613

avg / total       0.77      0.77      0.77      4211



In [25]:
vectorizer = TfidfVectorizer(min_df=2,
                             use_idf=True,
                             stop_words='english')
train_vectors = vectorizer.fit_transform(train.ix[:,4].tolist())
test_vectors = vectorizer.transform(test.ix[:,4].tolist())

base1 = svm.SVC(kernel='linear')
base1.fit(train_vectors, train_pol_y)
predict_base1 = base1.predict(test_vectors)

target_names = ["Negative","Positive"]
print classification_report(test_pol_y,predict_base1, target_names = target_names)

             precision    recall  f1-score   support

   Negative       0.80      0.85      0.82      2598
   Positive       0.72      0.65      0.68      1613

avg / total       0.77      0.77      0.77      4211



### Try LinearSVC

In [34]:
vectorizer = TfidfVectorizer(min_df=2,
                             use_idf=True,
                             stop_words='english')
train_vectors = vectorizer.fit_transform(train.ix[:,4].tolist())
test_vectors = vectorizer.transform(test.ix[:,4].tolist())

base1 = svm.LinearSVC()

C_options = {'C': np.arange(0.1, 1, 0.1)}
grid = GridSearchCV(base1, C_options)

grid.fit(train_vectors, train_pol_y)
preds = grid.predict(test_vectors)
    
# Output best param
print "Best value for C: %.2f" %grid.best_params_['C']
print "F1 score for Logistic Regression: %.3f" %metrics.f1_score(test_pol_y, preds, average="weighted") + "\n"


target_names = ["Negative","Positive"]
print classification_report(test_pol_y, preds, target_names = target_names)

Best value for C: 0.20
F1 score for Logistic Regression: 0.763

             precision    recall  f1-score   support

   Negative       0.78      0.86      0.82      2598
   Positive       0.73      0.62      0.67      1613

avg / total       0.76      0.77      0.76      4211



## Try other models

In [27]:
def P3(train_data, train_labels, dev_data, dev_labels):
### STUDENT START ###

    # Data setup
    vec = TfidfVectorizer(min_df=2,
                             use_idf=True,
                             stop_words='english')
    train_mat = vec.fit_transform(train_data)    
    dev_fit = vec.transform(dev_data)
    
    # K-NN MODEL
#     knn = KNeighborsClassifier()
    
    # Use GridSearchCV to find optimal k value
#     k_options = {'n_neighbors':[i for i in range(1, 3)]}
#     knn_grid = GridSearchCV(knn, k_options)

#     knn_grid.fit(train_mat, train_labels)
#     preds = knn_grid.predict(dev_fit)
    
    # Output best param
#     print "Best value for k: %d" %knn_grid.best_params_['n_neighbors']
#     print "F1 score for K-NN: %.3f" %metrics.f1_score(dev_labels, preds, average="weighted") + "\n"
    
    # MULTINOMIAL NAIVE BAYES
    # Repeat process for multinomial Naive Bayes
    mul = MultinomialNB(alpha=0.5)
    
    alpha_options = {'alpha': np.arange(0.01, 1, 0.01)}
    mul_grid = GridSearchCV(mul, alpha_options)
    
    mul_grid.fit(train_mat, train_labels)
    mul_preds = mul_grid.predict(dev_fit)
    
    # Output best param
    print "Best value for alpha: %.2f" %mul_grid.best_params_['alpha']
    print "F1 score for Multinomial Naive Bayes: %.3f" %metrics.f1_score(dev_labels, mul_preds, average="weighted") + "\n"

    
    # LOGISTIC REGRESSION
    # Repeat process for multinomial Naive Bayes
    log = LogisticRegression()
#     log = LogisticRegression(class_weight='balanced')
    
    C_options = {'C': np.arange(0.1, 1, 0.1)}
    log_grid = GridSearchCV(log, C_options)

    log_grid.fit(train_mat, train_labels)
    log_preds = log_grid.predict(dev_fit)
    
    # Output best param
    print "Best value for C: %.2f" %log_grid.best_params_['C']
    print "F1 score for Logistic Regression: %.3f" %metrics.f1_score(dev_labels, log_preds, average="weighted") + "\n"
    
    # Output sum of squared weights for a series of values of C
#     C_vals = np.arange(0.1, 1, 0.1)
#     for val in C_vals:
#         log_mod = LogisticRegression(C=val)
#         log_mod.fit(train_mat, train_labels)
#         print "Sum of squared weights for C=%.2f:" %val, np.square(log_mod.coef_).sum(axis=1)
    

### STUDENT END ###

P3(train_pol_x, train_pol_y, test_pol_x, test_pol_y)

Best value for alpha: 0.79
F1 score for Multinomial Naive Bayes: 0.752

Best value for C: 0.70
F1 score for Logistic Regression: 0.758



In [18]:
y = np.asarray(train_pol_y)
y.shape

(16840,)